In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Reading the Dataset
dataset = pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,True,False,False,False,False,False,True,False,False,True
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,True,False,False,True,True,False,True,False,True,True
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,True,False,False,True,True,False,False,False,False,True
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,True,False,False,True,True,False,True,False,True,True


In [2]:
dataset["classification_yes"].value_counts()

classification_yes
True     249
False    150
Name: count, dtype: int64

In [3]:
indep=dataset[["age","bp","al","su","bgr","bu","sc","sod","pot","hrmo","pcv","wc","rc","sg_b","sg_c","sg_e","rbc_normal","pc_normal","pcc_present","ba_present","htn_yes","dm_yes","cad_yes","appet_yes","pe_yes","ane_yes"]]
dep=dataset["classification_yes"]

In [4]:
indep.shape

(399, 26)

In [5]:
dep

0       True
1       True
2       True
3       True
4       True
       ...  
394     True
395     True
396     True
397     True
398    False
Name: classification_yes, Length: 399, dtype: bool

In [6]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\gssan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gssan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gssan\anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\gssan\anaconda3\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [9]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [10]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'random'}: 0.9550425208319945


In [11]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[49  2]
 [ 4 78]]


In [12]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

       False       0.92      0.96      0.94        51
        True       0.97      0.95      0.96        82

    accuracy                           0.95       133
   macro avg       0.95      0.96      0.95       133
weighted avg       0.96      0.95      0.96       133



In [13]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


np.float64(0.9560019129603061)

In [14]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001057,0.000030,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000941,0.000217,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.002249,0.000293,0.003641,0.000767,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.907846,0.943041,0.925524,0.943651,0.943651,0.932743,0.014255,7
3,0.002201,0.000497,0.003520,0.000431,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.962963,0.944161,0.943651,0.943093,0.962264,0.951226,0.009306,3
4,0.002040,0.000236,0.003331,0.000391,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.963284,0.981014,0.962573,0.905155,0.885478,0.939501,0.037200,6
5,0.001861,0.000197,0.003530,0.000560,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.890467,0.981014,0.962573,0.981217,0.962264,0.955507,0.033578,2
6,0.000760,0.000036,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000780,0.000086,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.001885,0.000153,0.003216,0.000061,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.944707,0.903610,0.888515,0.942332,0.981031,0.932039,0.032769,8
9,0.001641,0.000011,0.003362,0.000344,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.000000,0.981014,1.000000,0.962264,0.942332,0.977122,0.022330,1


In [18]:
age_input=float(input("Age:"))
bp=float(input("bp:"))
al=int(input("al:"))
su=int(input("su:"))
bgr=int(input("bgr:"))
bu=int(input("bu:"))
sc=float(input("sc:"))
sod=float(input("sod:"))
pot=float(input("pot:"))
hrmo=float(input("hrmo:"))
pcv=float(input("pcv:"))
wc=float(input("wc:"))
rc=float(input("rc:"))
sg_b=int(input("sg_b:0 or 1"))
sg_c=int(input("sg_c: 0 or 1"))
sg_e=int(input("sg_e: 0 or 1"))
rbc_normal=int(input("rbc_normal: 0 or 1"))
pc_normal=int(input("pc_normal:0 or 1"))
pcc_present=int(input("pcc_present:0 or 1"))
ba_present=int(input("ba_present:0 or 1"))
htn_yes=int(input("htn_yes:0 or 1"))
dm_yes=int(input("dm_yes:0 or 1"))
cad_yes=int(input("cad_yes: 0 or 1:"))
appet_yes=int(input("appet_yes: 0 or 1:"))
pe_yes=int(input("pe_yes: 0 or 1:"))
ane_yes=int(input("ane_yes: 0 or 1:"))

Age: 41
bp: 122
al: 3
su: 2
bgr: 219
bu: 115
sc: 1.3
sod: 134
pot: 4.4
hrmo: 12
pcv: 34
wc: 8600
rc: 4.8
sg_b:0 or 1 1
sg_c: 0 or 1 0
sg_e: 0 or 1 1
rbc_normal: 0 or 1 1
pc_normal:0 or 1 1
pcc_present:0 or 1 1
ba_present:0 or 1 1
htn_yes:0 or 1 1
dm_yes:0 or 1 0
cad_yes: 0 or 1: 1
appet_yes: 0 or 1: 0
pe_yes: 0 or 1: 1
ane_yes: 0 or 1: 1


In [19]:
Future_Prediction=grid.predict([[age_input,bp,al,su,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc,sg_b,sg_c,sg_e,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,pe_yes]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[ True]


In [20]:
import pickle
filename="finalized_model_DT_Classification.sav"
pickle.dump(clf_report,open(filename,'wb'))